In [97]:
# Imports
import pandas as pd
import sys
import glob
import errno
import csv
import numpy as np
from nltk.corpus import stopwords
import re
import nltk.data
import nltk
import os
from collections import OrderedDict
from subprocess import check_call
from shutil import copyfile
from sklearn.metrics import log_loss
%matplotlib inline
import matplotlib.pyplot as plt
import mpld3
mpld3.enable_notebook()
import seaborn as sns
from collections import Counter
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import ensemble, metrics, model_selection, naive_bayes
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from nltk.corpus import stopwords

In [98]:
# Read data
train = "../data/train.csv"
test = "../data/test.csv"
wv = "../../../../glove.6B/glove.6B.50d.txt"
X_train = pd.read_csv( train, header=0,delimiter="," )
X_test = pd.read_csv( test, header=0,delimiter="," )

word_vecs = {}
with open(wv) as f:
    for line in f:
       vals = line.split()
       word_vecs[vals[0]] = np.array(vals[1::],dtype=float)
authors = ['EAP','MWS','HPL']

Y_train = LabelEncoder().fit_transform(X_train['author'])

In [99]:
# Clean data
def getWordVectors(X_train,X_test,word_vecs):
    X_train['word_vectors'] = [ [ word_vecs[word] for word in sentence if word in word_vecs] for sentence in X_train['text']]
    X_test['word_vectors'] = [ [ word_vecs[word] for word in sentence if word in word_vecs] for sentence in X_test['text']] 
    return X_train,X_test

def getSentenceVectors(X_train,X_test):
    X_train['sentence_vectors'] =[np.mean(sentence,axis = 0) for sentence in X_train['word_vectors']]
    X_test['sentence_vectors'] =[np.mean(sentence,axis = 0) for sentence in X_test['word_vectors']] 
    return X_train,X_test

def clean(X_train,X_test):
    X_train['words'] = [re.sub("[^a-zA-Z]"," ", data).lower().split() for data in X_train['text']]
    X_test['words'] = [re.sub("[^a-zA-Z]"," ", data).lower().split() for data in X_test['text']]
    return X_train,X_test
X_train,X_test = clean(X_train,X_test)
X_train,X_test = getWordVectors(X_train,X_test,word_vecs)
X_train,X_test = getSentenceVectors(X_train,X_test)
#X_train.head()

In [100]:
# Feature Engineering
# Punctuation
punctuations = [{"id":1,"p":"[;:]"},{"id":2,"p":"[,.]"},{"id":3,"p":"[?]"},{"id":4,"p":"[\']"},{"id":5,"p":"[\"]"},{"id":6,"p":"[;:,.?\'\"]"}]
for p in punctuations:
    punctuation = p["p"]
    _train =  [ sentence.split() for sentence in X_train['text'] ]
    X_train['punc_'+str(p["id"])] = [len([word for word in sentence if bool(re.search(punctuation, word))])*100.0/len(sentence) for sentence in _train]    

    _test =  [ sentence.split() for sentence in X_test['text'] ]
    X_test['punc_'+str(p["id"])] = [len([word for word in sentence if bool(re.search(punctuation, word))])*100.0/len(sentence) for sentence in _test]    



In [101]:
# Feature Engineering
# Stop Words
_dist_train = [x for x in X_train['words']]
X_train['stop_word'] = [len([word for word in sentence if word in stopwords.words('english')])*100.0/len(sentence) for sentence in _dist_train]

_dist_test = [x for x in X_test['words']]
X_test['stop_word'] = [len([word for word in sentence if word in stopwords.words('english')])*100.0/len(sentence) for sentence in _dist_test]    

In [102]:
# Feature Engineering
# tfidf - words - nb+svd
def tfidfWords(X_train,X_test):
    tfidf_vec = TfidfVectorizer(stop_words='english', ngram_range=(1,3))
    full_tfidf = tfidf_vec.fit_transform(X_train['text'].values.tolist() + X_test['text'].values.tolist())
    train_tfidf = tfidf_vec.transform(X_train['text'].values.tolist())
    test_tfidf = tfidf_vec.transform(X_test['text'].values.tolist())
    return train_tfidf,test_tfidf,full_tfidf
    
def runMNB(train_X, train_y, test_X, test_y, test_X2):
    model = naive_bayes.MultinomialNB()
    model.fit(train_X, train_y)
    pred_test_y = model.predict_proba(test_X)
    pred_test_y2 = model.predict_proba(test_X2)
    return pred_test_y, pred_test_y2, model

def runSVD(full_tfidf,train_tfidf,test_tfidf):   
    n_comp = 20
    svd_obj = TruncatedSVD(n_components=n_comp)
    svd_obj.fit(full_tfidf)
    train_svd = pd.DataFrame(svd_obj.transform(train_tfidf))
    test_svd = pd.DataFrame(svd_obj.transform(test_tfidf))

    train_svd.columns = ['svd_word_'+str(i) for i in range(n_comp)]
    test_svd.columns = ['svd_word_'+str(i) for i in range(n_comp)]
    return train_svd,test_svd

def do_tfidf_MNB(X_train,X_test,Y_train):
    train_tfidf,test_tfidf,full_tfidf = tfidfWords(X_train,X_test)
    cv_scores = []
    pred_full_test = 0
    pred_train = np.zeros([X_train.shape[0], 3])
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
    for dev_index, val_index in kf.split(X_train):
        dev_X, val_X = train_tfidf[dev_index], train_tfidf[val_index]
        dev_y, val_y = Y_train[dev_index], Y_train[val_index]
        pred_val_y, pred_test_y, model = runMNB(dev_X, dev_y, val_X, val_y, test_tfidf)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index,:] = pred_val_y
        cv_scores.append(metrics.log_loss(val_y, pred_val_y))
    print("Mean cv score : ", np.mean(cv_scores))
    pred_full_test = pred_full_test / 5.
    return pred_train,pred_full_test

def do_tfidf_SVD(X_train,X_test,Y_train):
    train_tfidf,test_tfidf,full_tfidf = tfidfWords(X_train,X_test)
    train_svd,test_svd = runSVD(full_tfidf,train_tfidf,test_tfidf)
    return train_svd,test_svd

pred_train,pred_test = do_tfidf_MNB(X_train,X_test,Y_train)
X_train["tfidf_words_nb_eap"] = pred_train[:,0]
X_train["tfidf_words_nb_hpl"] = pred_train[:,1]
X_train["tfidf_words_nb_mws"] = pred_train[:,2]
X_test["tfidf_words_nb_eap"] = pred_test[:,0]
X_test["tfidf_words_nb_hpl"] = pred_test[:,1]
X_test["tfidf_words_nb_mws"] = pred_test[:,2]

# pred_train,pred_test = do_tfidf_SVD(X_train,X_test,Y_train)
# print pred_train
# # X_train["tfidf_words_nb_eap"] = pred_train[:,0]
# # X_train["tfidf_words_nb_hpl"] = pred_train[:,1]
# # X_train["tfidf_words_nb_mws"] = pred_train[:,2]
# # X_test["tfidf_words_nb_eap"] = pred_test[:,0]
# # X_test["tfidf_words_nb_hpl"] = pred_test[:,1]
# # X_test["tfidf_words_nb_mws"] = pred_test[:,2]

('Mean cv score : ', 0.84221619836128525)


In [9]:
# Feature Engineering
# tfidf - chars - nb+svd
def tfidfWords(X_train,X_test):
    tfidf_vec = TfidfVectorizer(stop_words='english', ngram_range=(1,5),analyzer='char')
    full_tfidf = tfidf_vec.fit_transform(X_train['text'].values.tolist() + X_test['text'].values.tolist())
    train_tfidf = tfidf_vec.transform(X_train['text'].values.tolist())
    test_tfidf = tfidf_vec.transform(X_test['text'].values.tolist())
    return train_tfidf,test_tfidf
    
def runMNB(train_X, train_y, test_X, test_y, test_X2):
    model = naive_bayes.MultinomialNB()
    model.fit(train_X, train_y)
    pred_test_y = model.predict_proba(test_X)
    pred_test_y2 = model.predict_proba(test_X2)
    return pred_test_y, pred_test_y2, model

def do(X_train,X_test,Y_train):
    train_tfidf,test_tfidf = tfidfWords(X_train,X_test)
    cv_scores = []
    pred_full_test = 0
    pred_train = np.zeros([X_train.shape[0], 3])
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
    for dev_index, val_index in kf.split(X_train):
        dev_X, val_X = train_tfidf[dev_index], train_tfidf[val_index]
        dev_y, val_y = Y_train[dev_index], Y_train[val_index]
        pred_val_y, pred_test_y, model = runMNB(dev_X, dev_y, val_X, val_y, test_tfidf)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index,:] = pred_val_y
        cv_scores.append(metrics.log_loss(val_y, pred_val_y))
    print("Mean cv score : ", np.mean(cv_scores))
    pred_full_test = pred_full_test / 5.
    return pred_train,pred_full_test
pred_train,pred_test = do(X_train,X_test,Y_train)
X_train["tfidf_chars_nb_eap"] = pred_train[:,0]
X_train["tfidf_chars_nb_hpl"] = pred_train[:,1]
X_train["tfidf_chars_nb_mws"] = pred_train[:,2]
X_test["tfidf_chars_nb_eap"] = pred_test[:,0]
X_test["tfidf_chars_nb_hpl"] = pred_test[:,1]
X_test["tfidf_chars_nb_mws"] = pred_test[:,2]

('Mean cv score : ', 0.7904152589474216)


In [103]:
# Feature Engineering
# count - words - nb
def countWords(X_train,X_test):
    count_vec = CountVectorizer(stop_words='english', ngram_range=(1,3))
    count_vec.fit(X_train['text'].values.tolist() + X_test['text'].values.tolist())
    train_count = count_vec.transform(X_train['text'].values.tolist())
    test_count = count_vec.transform(X_test['text'].values.tolist())
    return train_count,test_count
    
def runMNB(train_X, train_y, test_X, test_y, test_X2):
    model = naive_bayes.MultinomialNB()
    model.fit(train_X, train_y)
    pred_test_y = model.predict_proba(test_X)
    pred_test_y2 = model.predict_proba(test_X2)
    return pred_test_y, pred_test_y2, model

def do_count_MNB(X_train,X_test,Y_train):
    train_count,test_count=countWords(X_train,X_test)
    cv_scores = []
    pred_full_test = 0
    pred_train = np.zeros([X_train.shape[0], 3])
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
    for dev_index, val_index in kf.split(X_train):
        dev_X, val_X = train_count[dev_index], train_count[val_index]
        dev_y, val_y = Y_train[dev_index], Y_train[val_index]
        pred_val_y, pred_test_y, model = runMNB(dev_X, dev_y, val_X, val_y, test_count)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index,:] = pred_val_y
        cv_scores.append(metrics.log_loss(val_y, pred_val_y))
    print("Mean cv score : ", np.mean(cv_scores))
    pred_full_test = pred_full_test / 5.
    return pred_train,pred_full_test

pred_train,pred_test = do_count_MNB(X_train,X_test,Y_train)
X_train["count_words_nb_eap"] = pred_train[:,0]
X_train["count_words_nb_hpl"] = pred_train[:,1]
X_train["count_words_nb_mws"] = pred_train[:,2]
X_test["count_words_nb_eap"] = pred_test[:,0]
X_test["count_words_nb_hpl"] = pred_test[:,1]
X_test["count_words_nb_mws"] = pred_test[:,2]

('Mean cv score : ', 0.45091841616567468)


In [104]:
# Feature Engineering
# count - chars - nb
def countChars(X_train,X_test):
    count_vec = CountVectorizer(ngram_range=(1,7),analyzer='char')
    count_vec.fit(X_train['text'].values.tolist() + X_test['text'].values.tolist())
    train_count = count_vec.transform(X_train['text'].values.tolist())
    test_count = count_vec.transform(X_test['text'].values.tolist())
    return train_count,test_count
    
def runMNB(train_X, train_y, test_X, test_y, test_X2):
    model = naive_bayes.MultinomialNB()
    model.fit(train_X, train_y)
    pred_test_y = model.predict_proba(test_X)
    pred_test_y2 = model.predict_proba(test_X2)
    return pred_test_y, pred_test_y2, model

def do_count_chars_MNB(X_train,X_test,Y_train):
    train_count,test_count=countChars(X_train,X_test)
    cv_scores = []
    pred_full_test = 0
    pred_train = np.zeros([X_train.shape[0], 3])
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
    for dev_index, val_index in kf.split(X_train):
        dev_X, val_X = train_count[dev_index], train_count[val_index]
        dev_y, val_y = Y_train[dev_index], Y_train[val_index]
        pred_val_y, pred_test_y, model = runMNB(dev_X, dev_y, val_X, val_y, test_count)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index,:] = pred_val_y
        cv_scores.append(metrics.log_loss(val_y, pred_val_y))
    print("Mean cv score : ", np.mean(cv_scores))
    pred_full_test = pred_full_test / 5.
    return pred_train,pred_full_test

pred_train,pred_test = do_count_chars_MNB(X_train,X_test,Y_train)
X_train["count_chars_nb_eap"] = pred_train[:,0]
X_train["count_chars_nb_hpl"] = pred_train[:,1]
X_train["count_chars_nb_mws"] = pred_train[:,2]
X_test["count_chars_nb_eap"] = pred_test[:,0]
X_test["count_chars_nb_hpl"] = pred_test[:,1]
X_test["count_chars_nb_mws"] = pred_test[:,2]

('Mean cv score : ', 3.750763922681903)


In [105]:
# Word Embeddings
# this function creates a normalized vector for the whole sentence
#X_train['sentence_vectors'][0]

#foo.columns = ['svd_char_'+str(i) for i in range(n_comp)]
X_train = pd.concat([X_train,pd.DataFrame(X_train['sentence_vectors'].tolist())],axis=1)
X_test = pd.concat([X_test,pd.DataFrame(X_test['sentence_vectors'].tolist())],axis=1)

# xtrain_glove = [sent2vec(x) for x in tqdm(xtrain)]
# xvalid_glove = [sent2vec(x) for x in tqdm(xvalid)]
# xtrain_glove = np.array(xtrain_glove)
# xvalid_glove = np.array(xvalid_glove)

In [106]:
X_train[['sent_vec_'+str(i) for i in range(50)]] = pd.DataFrame(X_train['sentence_vectors'].tolist())
X_test[['sent_vec_'+str(i) for i in range(50)]] = pd.DataFrame(X_test['sentence_vectors'].tolist())

Index([u'id', u'text', u'words', u'word_vectors', u'sentence_vectors',
       u'punc_1', u'punc_2', u'punc_3', u'punc_4', u'punc_5', u'punc_6',
       u'stop_word', u'tfidf_words_nb_eap', u'tfidf_words_nb_hpl',
       u'tfidf_words_nb_mws', u'tfidf_chars_nb_eap', u'tfidf_chars_nb_hpl',
       u'tfidf_chars_nb_mws', u'count_chars_nb_eap', u'count_chars_nb_hpl',
       u'count_chars_nb_mws', u'count_words_nb_eap', u'count_words_nb_hpl',
       u'count_words_nb_mws', u'sent_vec_0', u'sent_vec_1', u'sent_vec_2',
       u'sent_vec_3', u'sent_vec_4', u'sent_vec_5', u'sent_vec_6',
       u'sent_vec_7', u'sent_vec_8', u'sent_vec_9', u'sent_vec_10',
       u'sent_vec_11', u'sent_vec_12', u'sent_vec_13', u'sent_vec_14',
       u'sent_vec_15', u'sent_vec_16', u'sent_vec_17', u'sent_vec_18',
       u'sent_vec_19', u'sent_vec_20', u'sent_vec_21', u'sent_vec_22',
       u'sent_vec_23', u'sent_vec_24', u'sent_vec_25', u'sent_vec_26',
       u'sent_vec_27', u'sent_vec_28', u'sent_vec_29', u'sent_vec_3

In [ ]:
# LSTM
# scale the data before any neural net:
scl = preprocessing.StandardScaler()
xtrain_glove_scl = scl.fit_transform(xtrain_glove)
xvalid_glove_scl = scl.transform(xvalid_glove)

# we need to binarize the labels for the neural net
ytrain_enc = np_utils.to_categorical(ytrain)
yvalid_enc = np_utils.to_categorical(yvalid)

# create a simple 3 layer sequential neural net
model = Sequential()

model.add(Dense(300, input_dim=300, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(300, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Dense(3))
model.add(Activation('softmax'))

# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.fit(xtrain_glove_scl, y=ytrain_enc, batch_size=64, 
          epochs=5, verbose=1, 
          validation_data=(xvalid_glove_scl, yvalid_enc))

# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 70

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

# zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        
# A simple LSTM with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, verbose=1, validation_data=(xvalid_pad, yvalid_enc))

In [107]:
# Model
# XGBoost
def runXGB(train_X, train_y, test_X, test_y=None, test_X2=None, seed_val=0, child=1, colsample=0.3):
    param = {}
    param['objective'] = 'multi:softprob'
    param['eta'] = 0.1
    param['max_depth'] = 3
    param['silent'] = 1
    param['num_class'] = 3
    param['eval_metric'] = "mlogloss"
    param['min_child_weight'] = child
    param['subsample'] = 0.8
    param['colsample_bytree'] = colsample
    param['seed'] = seed_val
    num_rounds = 2000

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=50, verbose_eval=20)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)

    pred_test_y = model.predict(xgtest, ntree_limit = model.best_ntree_limit)
    if test_X2 is not None:
        xgtest2 = xgb.DMatrix(test_X2)
        pred_test_y2 = model.predict(xgtest2, ntree_limit = model.best_ntree_limit)
    return pred_test_y, pred_test_y2, model

def do(X_train,X_test,Y_train):
    drop_columns=["id","text","words","word_vectors","sentence_vectors"]
    x_train = X_train.drop(drop_columns+['author'],axis=1)
    x_test = X_test.drop(drop_columns,axis=1)
    y_train = Y_train
    
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
    cv_scores = []
    pred_full_test = 0
    pred_train = np.zeros([x_train.shape[0], 3])
    for dev_index, val_index in kf.split(x_train):
        dev_X, val_X = x_train.loc[dev_index], x_train.loc[val_index]
        dev_y, val_y = y_train[dev_index], y_train[val_index]
        pred_val_y, pred_test_y, model = runXGB(dev_X, dev_y, val_X, val_y, x_test, seed_val=0, colsample=0.7)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index,:] = pred_val_y
        cv_scores.append(metrics.log_loss(val_y, pred_val_y))
    print("cv scores : ", cv_scores)
    return pred_full_test/5
result = do(X_train,X_test,Y_train)

[0]	train-mlogloss:1.00235	test-mlogloss:1.00238
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
[20]	train-mlogloss:0.414865	test-mlogloss:0.419062
[40]	train-mlogloss:0.349226	test-mlogloss:0.360598
[60]	train-mlogloss:0.330105	test-mlogloss:0.349934
[80]	train-mlogloss:0.31826	test-mlogloss:0.346674
[100]	train-mlogloss:0.309008	test-mlogloss:0.344737
[120]	train-mlogloss:0.300735	test-mlogloss:0.343988
[140]	train-mlogloss:0.293307	test-mlogloss:0.342923
[160]	train-mlogloss:0.285641	test-mlogloss:0.342578
[180]	train-mlogloss:0.277332	test-mlogloss:0.342061
[200]	train-mlogloss:0.270671	test-mlogloss:0.342089
[220]	train-mlogloss:0.263426	test-mlogloss:0.342538
Stopping. Best iteration:
[185]	train-mlogloss:0.276056	test-mlogloss:0.341867

[0]	train-mlogloss:1.00264	test-mlogloss:1.00308
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Wil

In [108]:
# Write Results

def writeResult(result,test):
    # count number of files
    path, dirs, files = os.walk("../results").next()
    file_count = len(files)/2+1

    # Write the test results
    data=OrderedDict()
    data["id"]=test["id"] 
    data["EAP"]=result[0]#["EAP"]
    data["HPL"]=result[1]#["HPL"]	
    data["MWS"]=result[2]#["MWS"]
    output = pd.DataFrame(data=data)
    filename = "../results/result"+str(file_count)+".csv"
    output.to_csv( filename, index=False )
    filename = "../results/result"+str(file_count)+"compr.csv"
    output.to_csv( filename, index=False )
    check_call(['gzip', filename])

writeResult(result.T,X_test)

In [140]:
tfidf_vec = TfidfVectorizer(ngram_range=(1,3))
full_tfidf = tfidf_vec.fit_transform(X_train['text'].values.tolist() + X_test['text'].values.tolist())
train_tfidf = tfidf_vec.transform(X_train['text'].values.tolist())
test_tfidf = tfidf_vec.transform(X_test['text'].values.tolist())

In [141]:
feature_names = tfidf_vec.get_feature_names()
corpus_index = [n for n in X_train['words']]
import pandas as pd
df = pd.DataFrame(train_tfidf.T.todense(), index=feature_names)


0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
5        0.0
6        0.0
7        0.0
8        0.0
9        0.0
10       0.0
11       0.0
12       0.0
13       0.0
14       0.0
15       0.0
16       0.0
17       0.0
18       0.0
19       0.0
20       0.0
21       0.0
22       0.0
23       0.0
24       0.0
25       0.0
26       0.0
27       0.0
28       0.0
29       0.0
        ... 
19549    0.0
19550    0.0
19551    0.0
19552    0.0
19553    0.0
19554    0.0
19555    0.0
19556    0.0
19557    0.0
19558    0.0
19559    0.0
19560    0.0
19561    0.0
19562    0.0
19563    0.0
19564    0.0
19565    0.0
19566    0.0
19567    0.0
19568    0.0
19569    0.0
19570    0.0
19571    0.0
19572    0.0
19573    0.0
19574    0.0
19575    0.0
19576    0.0
19577    0.0
19578    0.0
Name: brown, Length: 19579, dtype: float64